In [ ]:
%run lta_datamall_data.ipynb

In [ ]:
%run nearest_mrt_station_to_bus_stops.ipynb

In [ ]:
# mrt stations geodataframe
import os
os.environ['OGR_GEOMETRY_ACCEPT_UNCLOSED_RING'] = 'NO'

mrt_stations_gdf = geopandas.read_file('./TrainStation_Jul2024/RapidTransitSystemStation.shp')
mrt_stations_gdf_3857 = mrt_stations_gdf.to_crs(3857)
mrt_stations_gdf_3857

In [ ]:
mrt_stations_gdf2 = geopandas.read_file('./mrt_stations/mrt_stations.shp')

mrt_stations_gdf2["lat_lng"] = geopandas.points_from_xy(mrt_stations_gdf2["LATITUDE"], mrt_stations_gdf2["LONGITUDE"])
mrt_stations_gdf2["line"] = mrt_stations_gdf2["CODE"].str.slice(0, 2) 

mrt_lines_code = ["NS", "EW", "CC", "NE", "TE", "DT"]

mrt_gdf_wo_lrt = mrt_stations_gdf2[mrt_stations_gdf2["line"].isin(mrt_lines_code)].reset_index()
mrt_gdf_wo_lrt = mrt_gdf_wo_lrt.drop("index", axis = 1)

mrt_gdf_wo_lrt["STN_NAM_DE"] = mrt_gdf_wo_lrt["BUILDING"].str.split('(').str[0].str.strip()
mrt_stations_3857_2 = mrt_gdf_wo_lrt.merge(mrt_stations_gdf_3857, how = "left", on = "STN_NAM_DE")

mrt_stations_3857_2

# Brown Line

In [ ]:
brown_line = ['WOODLANDS NORTH MRT STATION', 'WOODLANDS MRT STATION', 'WOODLANDS SOUTH MRT STATION',
              'SPRINGLEAF MRT STATION', 'LENTOR MRT STATION', 'MAYFLOWER MRT STATION',
              'BRIGHT HILL MRT STATION', 'UPPER THOMSON MRT STATION', 'CALDECOTT MRT STATION',
              'STEVENS MRT STATION', 'NAPIER MRT STATION', 'ORCHARD BOULEVARD MRT STATION',
              'ORCHARD MRT STATION', 'GREAT WORLD MRT STATION', 'HAVELOCK MRT STATION',
              'OUTRAM PARK MRT STATION', 'MAXWELL MRT STATION', 'SHENTON WAY MRT STATION',
              'MARINA BAY MRT STATION', 'GARDENS BY THE BAY MRT STATION', 'TANJONG RHU MRT STATION',
              'KATONG PARK MRT STATION', 'TANJONG KATONG MRT STATION', 'MARINE PARADE MRT STATION',
              'MARINE TERRACE MRT STATION', 'SIGLAP MRT STATION', 'BAYSHORE MRT STATION',
              'BEDOK SOUTH MRT STATION', 'SUNGEI BEDOK MRT STATION']

# filter brown line stations from mrt stations geodataframe
brown_line_stations_gdf = mrt_stations_gdf_3857[mrt_stations_gdf_3857['STN_NAM_DE'].isin(brown_line)]

# order brown line stations based on list
brown_line_stations_gdf['STN_NAM_DE'] = pd.Categorical(brown_line_stations_gdf['STN_NAM_DE'],
                                                       categories = brown_line,
                                                       ordered = True)
brown_line_stations_gdf = brown_line_stations_gdf.sort_values('STN_NAM_DE')
brown_line_stations_gdf


In [7]:
# ordered list of brown line station coordinates
brown_line_stations_gdf['coordinates'] = brown_line_stations_gdf['geometry'].centroid
brown_line_stations_gdf

# create a linestring for the brown line route
from shapely.geometry import LineString
brown_line_route = LineString(brown_line_stations_gdf['coordinates'])

In [ ]:
from shapely.geometry import LineString

# combine bus routes
bus_routes_combined = bus_routes_gdf_3857.groupby(['ServiceNo', 'Direction']).apply(lambda x: LineString(x.geometry.tolist())).reset_index()
bus_routes_combined.columns = ['ServiceNo', 'Direction', 'geometry']
bus_routes_combined

In [ ]:
# calculate the overlapping distance of each bus route and the brown line
buffer_distance = 150
buffered_brown_line = brown_line_route.buffer(buffer_distance)

overlap_distance_output = []

for i, route in tqdm(bus_routes_combined.iterrows()):
    overlap = buffered_brown_line.intersection(route.geometry)
    if overlap.is_empty:
        overlap_distance_output.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': 0,
            'MRT line': 'Thomson-East Coast Line'
            })
    else:
        overlap_distance_output.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': overlap.length,
            'MRT line': 'Thomson-East Coast Line'
            })

overlap_distance = pd.DataFrame(overlap_distance_output)
overlap_distance

In [ ]:
# filter bus routes with overlap
bus_routes_overlap = overlap_distance[overlap_distance['Overlap Distance'] > 0].sort_values(by = 'Overlap Distance', ascending = False)
bus_routes_overlap

# Blue Line

In [ ]:
blue_line = [
    'SUNGEI KADUT MRT STATION', 'BUKIT PANJANG MRT STATION', 'CASHEW MRT STATION', 'HILLVIEW MRT STATION', 'HUME MRT STATION', 'BEAUTY WORLD MRT STATION', 'KING ALBERT PARK MRT STATION', 
    'SIXTH AVENUE MRT STATION', 'TAN KAH KEE MRT STATION', 'BOTANIC GARDENS MRT STATION', 'STEVENS MRT STATION', 'NEWTON MRT STATION', 
    'LITTLE INDIA MRT STATION', 'ROCHOR MRT STATION', 'BUGIS MRT STATION', 'PROMENADE MRT STATION', 'BAYFRONT MRT STATION', 'DOWNTOWN MRT STATION', 'TELOK AYER MRT STATION', 
    'CHINATOWN MRT STATION', 'FORT CANNING MRT STATION', 'BENCOOLEN MRT STATION', 'JALAN BESAR MRT STATION', 'BENDEMEER MRT STATION', 
    'GEYLANG BAHRU MRT STATION', 'MATTAR MRT STATION', 'MACPHERSON MRT STATION', 'UBI MRT STATION', 'KAKI BUKIT MRT STATION', 'BEDOK NORTH MRT STATION', 
    'BEDOK RESERVOIR MRT STATION', 'TAMPINES WEST MRT STATION', 'TAMPINES MRT STATION', 'TAMPINES EAST MRT STATION', 'UPPER CHANGI MRT STATION', 'EXPO MRT STATION'
]

# filter brown line stations from mrt stations geodataframe
blue_line_stations_gdf = mrt_stations_gdf_3857[mrt_stations_gdf_3857['STN_NAM_DE'].isin(blue_line)]

# order brown line stations based on list
blue_line_stations_gdf['STN_NAM_DE'] = pd.Categorical(blue_line_stations_gdf['STN_NAM_DE'],
                                                       categories = blue_line,
                                                       ordered = True)
blue_line_stations_gdf = blue_line_stations_gdf.sort_values('STN_NAM_DE')
blue_line_stations_gdf



In [12]:
# ordered list of blue line station coordinates
blue_line_stations_gdf['coordinates'] = blue_line_stations_gdf['geometry'].centroid
blue_line_stations_gdf

# create a linestring for the blue line route
from shapely.geometry import LineString
blue_line_route = LineString(blue_line_stations_gdf['coordinates'])

In [ ]:
from shapely.geometry import LineString

# combine bus routes
bus_routes_combined = bus_routes_gdf_3857.groupby(['ServiceNo', 'Direction']).apply(lambda x: LineString(x.geometry.tolist())).reset_index()
bus_routes_combined.columns = ['ServiceNo', 'Direction', 'geometry']
bus_routes_combined

In [ ]:
# calculate the overlapping distance of each bus route and the blue line
buffer_distance = 150
buffered_blue_line = blue_line_route.buffer(buffer_distance)

overlap_distance_output = []

for i, route in tqdm(bus_routes_combined.iterrows()):
    overlap = buffered_blue_line.intersection(route.geometry)
    if overlap.is_empty:
        overlap_distance_output.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': 0,
            'MRT line': 'Downtown Line'
            })
    else:
        overlap_distance_output.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': overlap.length,
            'MRT line': 'Downtown Line'
            })

overlap_distance_blue = pd.DataFrame(overlap_distance_output)
overlap_distance_blue

In [ ]:
# filter bus routes with overlap with blue line
bus_routes_overlap_blue = overlap_distance_blue[overlap_distance_blue['Overlap Distance'] > 0].sort_values(by = 'Overlap Distance', ascending = False)
bus_routes_overlap_blue

In [16]:
bus_routes_overlap = pd.concat([bus_routes_overlap, bus_routes_overlap_blue], axis = 0)

# Purple Line

In [ ]:
purple_line = [
    'HARBOURFRONT MRT STATION',
    'OUTRAM PARK MRT STATION',
    'CHINATOWN MRT STATION',
    'CLARKE QUAY MRT STATION',
    'DHOBY GHAUT MRT STATION',
    'LITTLE INDIA MRT STATION',
    'FARRER PARK MRT STATION',
    'BOON KENG MRT STATION',
    'POTONG PASIR MRT STATION',
    'WOODLEIGH MRT STATION',
    'SERANGOON MRT STATION',
    'KOVAN MRT STATION',
    'HOUGANG MRT STATION',
    'BUANGKOK MRT STATION',
    'SENGKANG MRT STATION',
    'PUNGGOL MRT STATION'
]


# filter purple line stations from mrt stations geodataframe
purple_line_stations_gdf = mrt_stations_gdf_3857[mrt_stations_gdf_3857['STN_NAM_DE'].isin(purple_line)]

# order purple line stations based on list
purple_line_stations_gdf['STN_NAM_DE'] = pd.Categorical(purple_line_stations_gdf['STN_NAM_DE'],
                                                       categories = purple_line,
                                                       ordered = True)
purple_line_stations_gdf = purple_line_stations_gdf.sort_values('STN_NAM_DE')
purple_line_stations_gdf

In [18]:
# ordered list of purple line station coordinates
purple_line_stations_gdf['coordinates'] = purple_line_stations_gdf['geometry'].centroid
purple_line_stations_gdf

# create a linestring for the purple line route
from shapely.geometry import LineString
purple_line_route = LineString(purple_line_stations_gdf['coordinates'])

In [ ]:
from shapely.geometry import LineString

# combine bus routes
bus_routes_combined = bus_routes_gdf_3857.groupby(['ServiceNo', 'Direction']).apply(lambda x: LineString(x.geometry.tolist())).reset_index()
bus_routes_combined.columns = ['ServiceNo', 'Direction', 'geometry']
bus_routes_combined

In [ ]:
# calculate the overlapping distance of each bus route and the purple line
buffer_distance = 150
buffered_purple_line = purple_line_route.buffer(buffer_distance)

overlap_distance_output = []

for i, route in tqdm(bus_routes_combined.iterrows()):
    overlap = buffered_purple_line.intersection(route.geometry)
    if overlap.is_empty:
        overlap_distance_output.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': 0,
            'MRT line': 'North-East Line'
            })
    else:
        overlap_distance_output.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': overlap.length,
            'MRT line': 'North-East Line'
            })

overlap_distance_purple = pd.DataFrame(overlap_distance_output)
overlap_distance_purple

In [ ]:
# filter bus routes with overlap with purple line
bus_routes_overlap_purple = overlap_distance_purple[overlap_distance_purple['Overlap Distance'] > 0].sort_values(by = 'Overlap Distance', ascending = False)
bus_routes_overlap_purple

In [22]:
bus_routes_overlap = pd.concat([bus_routes_overlap, bus_routes_overlap_purple], axis = 0)

# Red Line

In [ ]:
red_line = mrt_stations_3857_2[mrt_stations_3857_2["line"] == "NS"]["STATION_NA"].tolist()

# filter brown line stations from mrt stations geodataframe
red_line_stations_gdf = mrt_stations_3857_2[mrt_stations_3857_2['line'] == "NS"]

red_line_stations_gdf["order"] = pd.to_numeric(red_line_stations_gdf["CODE"].str.slice(2, None))

# order brown line stations based on list
red_line_stations_gdf = red_line_stations_gdf.sort_values('order')
red_line_stations_gdf

In [ ]:
# ordered list of brown line station coordinates
red_line_stations_gdf['coordinates'] = red_line_stations_gdf['geometry_y'].centroid
red_line_stations_gdf


# create a linestring for the brown line route
from shapely.geometry import LineString
red_line_route = LineString(red_line_stations_gdf['coordinates'])
red_line_route

In [ ]:
buffered_red_line = red_line_route.buffer(buffer_distance)

overlap_distance_output_red = []

for i, route in tqdm(bus_routes_combined.iterrows()):
    overlap = buffered_red_line.intersection(route.geometry)
    if overlap.is_empty:
        overlap_distance_output_red.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': 0,
            'MRT line': 'North-South Line'
            })
    else:
        overlap_distance_output_red.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': overlap.length,
            'MRT line': 'North-South Line'
            })

overlap_distance_red = pd.DataFrame(overlap_distance_output_red)
overlap_distance_red

In [ ]:
bus_routes_overlap_red = overlap_distance_red[overlap_distance_red['Overlap Distance'] > 0].sort_values(by = 'Overlap Distance', ascending = False)
bus_routes_overlap_red

In [27]:
bus_routes_overlap = pd.concat([bus_routes_overlap, bus_routes_overlap_red], axis = 0)

In [ ]:
bus_routes_overlap

# Green line

In [ ]:
green_line = mrt_stations_3857_2[mrt_stations_3857_2["line"] == "EW"]["STATION_NA"].tolist()
green_line_stations_gdf = mrt_stations_3857_2[mrt_stations_3857_2['line'] == "EW"]

green_line_stations_gdf["order"] = pd.to_numeric(green_line_stations_gdf["CODE"].str.slice(2, None))

# order brown line stations based on list
green_line_stations_gdf = green_line_stations_gdf.sort_values('order')
green_line_stations_gdf

In [ ]:
green_line_stations_gdf = geopandas.GeoDataFrame(green_line_stations_gdf, geometry='geometry_y')

green_line_stations_gdf['coordinates'] = green_line_stations_gdf['geometry_y'].centroid
green_line_stations_gdf

# create a linestring for the green line route
from shapely.geometry import LineString
green_line_route = LineString(green_line_stations_gdf['coordinates'])
green_line_route

In [ ]:
buffered_green_line = green_line_route.buffer(buffer_distance)

overlap_distance_output_green = []

for i, route in tqdm(bus_routes_combined.iterrows()):
    overlap = buffered_green_line.intersection(route.geometry)
    if overlap.is_empty:
        overlap_distance_output_green.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': 0,
            'MRT line': 'East-West Line'
            })
    else:
        overlap_distance_output_green.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': overlap.length,
            'MRT line': 'East-West Line'
            })

overlap_distance_green = pd.DataFrame(overlap_distance_output_green)
overlap_distance_green

In [ ]:
bus_routes_overlap_green = overlap_distance_green[overlap_distance_green['Overlap Distance'] > 0].sort_values(by = 'Overlap Distance', ascending = False)
bus_routes_overlap_green

# Circle Line

In [ ]:
circle_line = mrt_stations_3857_2[mrt_stations_3857_2["line"] == "CC"]["STATION_NA"].tolist()

In [ ]:
circle_line_stations_gdf = mrt_stations_3857_2[mrt_stations_3857_2['line'] == "CC"]

circle_line_stations_gdf["order"] = pd.to_numeric(green_line_stations_gdf["CODE"].str.slice(2, None))

# order  stations based on list
circle_line_stations_gdf = circle_line_stations_gdf.sort_values('order')
circle_line_stations_gdf

In [ ]:
circle_line_stations_gdf = geopandas.GeoDataFrame(circle_line_stations_gdf, geometry='geometry_y')

circle_line_stations_gdf['coordinates'] = circle_line_stations_gdf['geometry_y'].centroid
circle_line_stations_gdf

# create a linestring for the green line route
from shapely.geometry import LineString
circle_line_route = LineString(circle_line_stations_gdf['coordinates'])
circle_line_route

In [ ]:
buffered_circle_line = circle_line_route.buffer(buffer_distance)

overlap_distance_output_circle = []

for i, route in tqdm(bus_routes_combined.iterrows()):
    overlap = buffered_circle_line.intersection(route.geometry)
    if overlap.is_empty:
        overlap_distance_output_circle.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': 0,
            'MRT line': 'Circle Line'
            })
    else:
        overlap_distance_output_circle.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': overlap.length,
            'MRT line': 'Circle Line'
            })

overlap_distance_circle = pd.DataFrame(overlap_distance_output_circle)
overlap_distance_circle

In [ ]:
bus_routes_overlap_circle = overlap_distance_circle[overlap_distance_circle['Overlap Distance'] > 0].sort_values(by = 'Overlap Distance', ascending = False)
bus_routes_overlap_circle